# Análise de dados eleições Camaragibe 2024
Esse pequeno projeto tem o objetivo de auxiliar no tratamento de dados na pesquisa eleitoral do município de Camaragibe 2024. O tratamento de dados era feito, diariamente, por meio de aplicações de filtros manuais em diversas planilhas, esse projeto tem objetivo de melhorar essa feitoria, automatizando a filtragem e tratamento desses dados referente as perguntas e respostas adequadas. Observe que o dataset usado é em formato .xlsx (excel), formato fornecido pelo ambiente da pesquisa.

## Base de dados
A base de dados usada é composta por cerca de 24 perguntas, posteriomente processadas, como resultados temos um resultado indicando a quantidade e porcentagem de cada variável do bloco relacionado a pesquisa.

In [1]:
# Bibliotecas e base de dados usada(xlsx)
import pandas as pd

# Carregar o DataFrame
df = pd.read_excel('src/CAMARAGIBE - INABI - 06.09.xlsx', sheet_name='DADOS')

# Sexo
contagem_sexo = df["1. SEXO"].value_counts()
valor_mais_frequente_sexo = contagem_sexo.idxmax()
df["1. SEXO"] = df["1. SEXO"].fillna(valor_mais_frequente_sexo)
porcentagem_sexo = (df["1. SEXO"].value_counts(normalize=True) * 100).round(2)
print(porcentagem_sexo)
print()

# Faixa Etária
faixa_etaria = df["2. FAIXA ETÁRIA"].value_counts()
valor_mais_frequente_faixa_etaria = faixa_etaria.idxmax()
df["2. FAIXA ETÁRIA"] = df["2. FAIXA ETÁRIA"].fillna(valor_mais_frequente_faixa_etaria)
porcentagem_faixa_etaria = (df["2. FAIXA ETÁRIA"].value_counts(normalize=True) * 100).round(2)
print(porcentagem_faixa_etaria)
print()


1. SEXO
FEMININO     58.93
MASCULINO    41.07
Name: proportion, dtype: float64

2. FAIXA ETÁRIA
45 A 59 ANOS       26.79
MAIS DE 70 ANOS    23.21
60 A 69 ANOS       16.07
25 A 34 ANOS       12.50
16 A 24 ANOS       12.50
35 A 44 ANOS        8.93
Name: proportion, dtype: float64

